# 0.데이터 로드 준비

In [ ]:
# function ClickConnect(){
#     console.log("코랩 연결 끊김 방지");
#     document.querySelector("colab-toolbar-button#connect").click()
# }
# click = setInterval(ClickConnect, 60 * 1000)

In [ ]:
# clearInterval(click);

In [ ]:
#  import shutil
#  shutil.rmtree('./data')

In [ ]:
pip install packages pytrends

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.2/516.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
flask 2.2.5 requires click>=8.0, but you have click 7.1.2 which is incompatible.
dask 2023.8.1 requires click>=8.0, but you have click 7.1.2 which is incompatible.
distributed 2023.8.1 requires click>=8.0, but 

In [ ]:
pip install dtw-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.0/745.0 kB 6.7 MB/s eta 0:00:00


In [ ]:
pip install packages google_trends_api

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
  Created wheel for google_trends_api: filename=google_trends_api-2.2.6-py3-none-any.whl size=6115 sha256=ec2b51400272e46982f380445c79088bf72b4437f6dbb697dcb36fe2c404ec12
  Stored in directory: /root/.cache/pip/wheels/59/38/e2/64aa41b0176045e727424fe15a0ac9d9f11048958012752943
Successfully built google_trends_api


In [ ]:
pip install 'urllib3<2'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2023.8.1 requires click>=8.0, but you have click 7.1.2 which is incompatible.


In [ ]:
import os
import sys
import time
import random
import datetime
import requests
import pandas as pd
import numpy as np
import hashlib, hmac, base64
from itertools import combinations, permutations
from dtw import *
import json
import urllib.request
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
import pickle
from pytz import timezone
from difflib import SequenceMatcher

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

from pytrends.request import TrendReq

g = globals()

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



## 1) API 설정

In [ ]:
def generate(timestamp, method, uri, secret_key):
    message = "{}.{}.{}".format(timestamp, method, uri)
    hash = hmac.new(bytes(secret_key, "utf-8"), bytes(message, "utf-8"), hashlib.sha256)
    #hash = hmac.new(secret_key.encode("utf-8"), message.encode("utf-8"), hashlib.sha256)
    hash.hexdigest()
    return base64.b64encode(hash.digest())

def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = generate(timestamp, method, uri, SECRET_KEY)
    return {'Content-Type': 'application/json; charset=UTF-8', 'X-Timestamp': timestamp, 'X-API-KEY': API_KEY, 'X-Customer': str(CUSTOMER_ID), 'X-Signature': signature}

def make_directory(path):
    os.makedirs(path, exist_ok=True)

In [ ]:
# api 설정
BASE_URL = 'https://api.searchad.naver.com'
CUSTOMER_ID = '2892384'
API_KEY = '01000000000c6fe5f2e0420440d9f675d55abc3a528ab077419121ed99e6ab152a8972b41f'
SECRET_KEY = 'AQAAAAAMb+Xy4EIEQNn2ddVavDpSXT4tzG4KaE6y88vvenBUVA=='
#
uri = '/keywordstool'  # BASE_URL 뒤에 오는 uri 입니다.
method = 'GET' # GET 방식을 사용합니다.

 # 1. 연관검색어 수집

## 1) 설정

In [ ]:
# 검색 키워드 리스트 로드 (py파일로 관리)
import main_keyword

# 날짜 설정
today = datetime.now(timezone('Asia/Seoul'))
today = today.strftime("%Y-%m-%d")
day = today[2:10].replace('-','')

# 결과 저장 폴더 생성
os.mkdir('./data')
os.mkdir('./data/rl_srch')
os.mkdir('./data/rl_srch/' + str(day))  # 키워드별 연관검색어 리스트 저장

## 2) 수집

In [ ]:
# 검색키워드 리스트
srch_keyword = ['keyword_final']

# 결과 저장 데이터프레임
tmp_df = pd.DataFrame()

# 검색어 수집 반복문
for word in srch_keyword :

    i = 1
    length = len(eval('main_keyword.' + f'{word}'))  # 코드 진행 상태 표시를 위한
    key_list = []                                    # 조건에 통과한 키워드 저장용

    print(f'{word} 연관검색어 추출 시작!!')

    for keyword in eval('main_keyword.' + f'{word}') :
        query = {'siteld':'', 'bixtpld':'', 'event' : '','month': '', 'showDetail':'1',
                'hintKeywords': f'{keyword}'}
        r = requests.get(
            BASE_URL +uri,
            params= query,
            headers = get_header(
                method = method,
                uri = uri,
                api_key = API_KEY,
                secret_key = SECRET_KEY,
                customer_id = CUSTOMER_ID))
        r_data = r.json()

        # 조건에 따라 데이터 추출
        df_naver = pd.DataFrame(r_data['keywordList'])
        # '< 10' 로 표기된 값들을 '9'로 변경
        df_naver = df_naver.replace('< 10', '9')
        # '< 10' 때문에 문자열로 인식된 컬럼 float으로 변경
        df_naver['monthlyPcQcCnt'] = df_naver['monthlyPcQcCnt'].astype('float64')
        df_naver['monthlyMobileQcCnt'] = df_naver['monthlyMobileQcCnt'].astype('float64')

        # 월간'총'검색수 생성
        df_naver.insert(loc=3, column='monthlyTotalCnt', value=df_naver['monthlyPcQcCnt'] + df_naver['monthlyMobileQcCnt'])
        # '월간총검색수'로 내림차순 정리
        df_naver = df_naver.sort_values('monthlyTotalCnt', ascending=False).reset_index(drop=True)
        # 컬럼명 한글로 변경
        df_naver = df_naver.rename(columns = {'relKeyword'              : '연관키워드',
                                            'monthlyPcQcCnt'          : '월간검색수_PC',
                                            'monthlyMobileQcCnt'      : '월간검색수_모바일',
                                            'monthlyTotalCnt'         : '월간검색수_합계',
                                            'monthlyAvePcClkCnt'      : '월평균클릭수_PC',
                                            'monthlyAveMobileClkCnt'  : '월평균클릭수_모바일',
                                            'monthlyAvePcCtr'         : '월평균클릭률_PC',
                                            'monthlyAveMobileCtr'     : '월평균클릭률_모바일',
                                            'plAvgDepth'              : '월평균노출광고수',
                                            'compIdx'                 : '경쟁정도'} )


        ## 테이블 저장 ##
        tmp_table = df_naver[['연관키워드','월간검색수_합계']].copy()

        if len(tmp_table.index) >= 0 :
            g[f'df_{keyword}'] =  tmp_table                                                                    # 검색어 연관검색어 정보 테이블
            tmp_table.to_csv('./data/rl_srch/' + str(day) + f'/{keyword}.csv',encoding='euc-kr', index=False)  # csv 형태로 저장
            key_list.append(keyword)
            tmp = pd.DataFrame(tmp_table.head(25))
            tmp_df = pd.concat([tmp_df, tmp])

        time.sleep(0.5)

        # 연관검색어 수집 진행률 표시
        if (i/length * 100) % 20 == 0 :
            print(f'{int(i/length * 100)}% complete!')
        if int(i/length * 100) % 100 == 0 :
            print('!! complete !!')
        i=i+1

keyword_final 연관검색어 추출 시작!!
20% complete!
40% complete!
60% complete!
80% complete!
100% complete!
!! complete !!


#  2. 키워드 트렌드 및 패턴 분석

In [ ]:
# 추세 데이터 불러오는 함수
## 필요한 인자 : 기준일자, 연관검색어, API ID, API PW, URL

def trend_load(std_time, rl_srch, id, pw, url) :

    df_total = pd.DataFrame()

    # 최근일자(=기준일자 하루전)
    end_time = std_time - relativedelta(days=1)
    end_time = end_time.strftime("%Y-%m-%d")
    # 시작일자(=4년전)
    start_time = std_time - relativedelta(years=4) - relativedelta(days=1)
    start_time = start_time.strftime("%Y-%m-%d")

    ## 시작일자, 끝일자, 검색어
    body = f'''<"startDate":"{start_time}",
            "endDate":"{end_time}",
            "timeUnit":"date",
            "keywordGroups":[<"groupName":"{rl_srch}","keywords":["{rl_srch}"]>]>'''

    body = body.replace('<','{')
    body = body.replace('>','}')


    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",id)
    request.add_header("X-Naver-Client-Secret",pw)
    request.add_header("Content-Type","application/json")

    response = urllib.request.urlopen(request, data=body.encode("utf-8"))
    rescode = response.getcode()


    if(rescode==200):
        response_body = response.read()
        response_data = response_body.decode('utf-8')
        # print(response_body.decode('utf-8'))
    else:
        print("Error Code:" + rescode)


    result = json.loads(response_data)

    date = [a['period'] for a in result['results'][0]['data']]
    ratio_data1 = [a['ratio'] for a in result['results'][0]['data']]

    df_lab = pd.DataFrame({'date' : date,
                            'groupName' : ratio_data1,
                            })
    if df_total.empty :
        df_total = df_lab
    else :
        df_total = pd.merge(df_total, df_lab, how='left',on='date')
    df_total = df_total.set_index('date')
    df_total = df_total.fillna(0)

    df_total.columns = [f'{rl_srch}']

    return df_total

## 1) 패턴 분류 함수

### (1) 일별 급상승 키워드 선별 함수

In [ ]:
# 일별 급상승 키워드 선별 함수

def daily_up(table, std_time) :

    # [기본 설정]
    # ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    # 최근일자(=기준일자 하루전)
    end_time = std_time - relativedelta(days=1)

    # 시작일자(=1년전)
    start_time = std_time - relativedelta(years=1) - relativedelta(days=1)

    ## 그래프 작성용 (=3년전)
    start_before = std_time - relativedelta(years=3) - relativedelta(days=1)

    # 형태 변화 (0000-00-00 형식)
    end_time = end_time.strftime("%Y-%m-%d")
    start_time = start_time.strftime("%Y-%m-%d")
    start_before = start_before.strftime("%Y-%m-%d")

    # 분석 기간 설정
    table.index = pd.to_datetime(table.index)
    table_tmp = table[(table.index >= start_time)&(table.index <= end_time)]
    table_graph = table[(table.index >= start_before)&(table.index <= end_time)]

    # 기간 내 데이터 공백 존재시 (최소 데이터 수 확보 목적)
    if len(table_tmp.index) < 350 :
        return None, None, None

    # 마지막 날짜가 어제 날짜인지 확인
    last_day = table.index[-1].strftime("%Y-%m-%d")
    if last_day != end_time :
      return None, None, None


    # 기간에 맞춰 상대적 검색량 수정
    table_tmp = table_tmp/table_tmp.max() * 100
    table_graph = table_graph/table_graph.max() * 100

    # 검색량 기준
    last = table_tmp.iloc[-1,0] # 가장 최근 일자 상대적 검색량
    last_2 = table_tmp.iloc[-2,0] # 가장 최근 일자 + 1일 상대적 검색량

    # 분산 기준
    var = np.var(table_tmp.iloc[:,0])

    # 최종 분석 테이블
    table_tmp_2 = table_tmp.copy()
    table_tmp_2.index = table_tmp_2.index.astype(str)
    table_tmp_2.index = [x[2:10].replace('-','/') for x in table_tmp_2.index]

    length = len(table_tmp_2.index)

    # 지표 (상승률)
    rise_rt = (last - last_2)/last_2 * 100
    rist_rt = round(rise_rt,2)

    # 그래프용 테이블 만들기
    result_graph = pd.DataFrame()
    result_graph['검색일자'] = table_graph.index
    result_graph['기준일자'] = today.strftime("%Y-%m-%d")
    result_graph['유형'] = '일별급상승'
    result_graph['연관검색어'] = table_graph.columns[0]
    result_graph['검색량'] = table_graph.values

    result_graph = result_graph[['기준일자','유형', '연관검색어', '검색일자', '검색량']]

    # [유형 분류]
    # ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    # 조건 적용
    if var > 200 :
        return None, None, None

    if (last > last_2 * 2.0) & (last >= 60) :
        print(f'일별 급상승 키워드 발견 : {table_graph.columns[0]}')

        return table_tmp_2, result_graph, rist_rt

    elif (last >= 95) & (last > last_2) :
        print(f'일별 급상승 키워드 발견 : {table_graph.columns[0]}')

        return table_tmp_2, result_graph, rist_rt

    elif (last_2 * 2.0 > 100) & (last >= 85) & (last > last_2) & ((last - last_2) > 5) :
        print(f'일별 급상승 키워드 발견 : {table_graph.columns[0]}')

        return table_tmp_2, result_graph, rist_rt

    else :
        return None, None, None


### (2) 주별 급상승 키워드 선별 함수

In [ ]:
# 주별 급상승 키워드 선별 함수

def weekly_up(table, std_time) :

    # [기본 설정]
    # ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    # 최근일자(=기준일자 하루전)
    end_time = std_time - relativedelta(days=1)

    # 시작일자(=2년전)
    start_time = std_time - relativedelta(years=2) - relativedelta(days=1)

    ## 그래프 작성용 (=3년전)
    start_before = std_time - relativedelta(years=3) - relativedelta(days=1)

    # 형태 변화
    end_time = end_time.strftime("%Y-%m-%d")
    start_time = start_time.strftime("%Y-%m-%d")
    start_before = start_before.strftime("%Y-%m-%d")

    # 분석 기간 설정
    table.index = pd.to_datetime(table.index)
    table_tmp = table[(table.index >= start_time)&(table.index <= end_time)]
    table_graph = table[(table.index >= start_before)&(table.index <= end_time)]

    # 기간 내 데이터 공백 존재시
    if len(table_tmp.index) < 700 :
        return None, None, None

    # 주별 데이터 테이블로 변환 (104주)
    tmp_list = []

    for i in range(0,104,1) :
        if i == 0 :
            days_7 = table_tmp.iloc[-(7*(i+1)) : , ].sum()
        else :
            days_7 = table_tmp.iloc[-(7*(i+1)) : -(7*(i)) , ].sum()
        tmp_list.append(days_7)
    tmp_frame = pd.DataFrame(tmp_list)

    r_idx = [i for i in range(tmp_frame.shape[0]-1,-1,-1)]
    result_tmp = pd.DataFrame(tmp_frame, index=r_idx)
    result_tmp.reset_index(drop=True, inplace=True)

    ## 그래프 작성용
    tmp_list_graph = []

    for j in range(0,156,1) :
        if j == 0 :
            days_7 = table_graph.iloc[-(7*(j+1)) : , ].sum()
        else :
            days_7 = table_graph.iloc[-(7*(j+1)) : -(7*(j)) , ].sum()
        tmp_list_graph.append(days_7)
    tmp_graph = pd.DataFrame(tmp_list_graph)

    r_idx_2 = [i for i in range(tmp_graph.shape[0]-1,-1,-1)]
    result_tmp_gph = pd.DataFrame(tmp_graph, index=r_idx_2)
    result_tmp_gph.reset_index(drop=True, inplace=True)


    # 기간에 맞춰 상대적 검색량 수정
    result_tmp = result_tmp/result_tmp.max() * 100
    result_tmp_gph = result_tmp_gph/result_tmp_gph.max() * 100

    # 인덱스 설정
    result_tmp.index = pd.date_range(end = f'{end_time}', periods = len(result_tmp.index), freq = '7d')
    result_tmp_gph.index = pd.date_range(end = f'{end_time}', periods = len(result_tmp_gph.index), freq = '7d')

    # 검색량 기준
    last = result_tmp.iloc[-1,0] # 가장 최근 일자 상대적 검색량
    last_2 = result_tmp.iloc[-2,0] # 가장 최근 일자 상대적 검색량

    # 분산 기준
    var = np.var(result_tmp.iloc[:,0])
    table_tmp_2 = result_tmp.copy()

    # 그래프용 테이블 만들기
    result_graph = pd.DataFrame()
    result_graph['검색일자'] = result_tmp_gph.index
    result_graph['기준일자'] = today.strftime("%Y-%m-%d")
    result_graph['유형'] = '주별급상승'
    result_graph['연관검색어'] = result_tmp_gph.columns[0]
    result_graph['검색량'] = result_tmp_gph.values

    result_graph = result_graph[['기준일자','유형', '연관검색어', '검색일자', '검색량']]

    # 지표 (추세성)
    rise_rt = (last - last_2)/last_2 * 100
    rist_rt = round(rise_rt,2)

    # [유형 분류]
    # ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    # 조건 적용
    if var > 350 :
        # print(var)
        return None, None, None

    if ((last >= last_2 * 2.0) & (last >= 60)) :
        print(f'주별 급상승 키워드 발견 : {table_graph.columns[0]}')

        return table_tmp_2, result_graph, rist_rt

    elif (last >= 95) & (last > last_2) :
        print(f'주별 급상승 키워드 발견 : {table_graph.columns[0]}')

        return table_tmp_2 , result_graph, rist_rt

    elif (last_2 * 2.0 > 100) & (last >= 90) & (last > last_2) & ((last - last_2) > 5) :
        print(f'주별 급상승 키워드 발견 : {table_graph.columns[0]}')

        return table_tmp_2 , result_graph, rist_rt
    else :
        return None, None, None


### (3) 주별 지속상승 키워드 선별 함수

In [ ]:
# 시계열 데이터 기울기 구하는 함수

def slop(dataframe) :
    df = dataframe.copy()
    df['time'] = range(1,len(df)+1)
    y = df.iloc[:,0]
    X = df['time']
    X = sm.add_constant(X)

    ols_fit = sm.OLS(y,X).fit()
    b1 = ols_fit.params.time

    return b1

In [ ]:
# 주별 지속상승 키워드 선별 함수

def weekly_stay(table, std_time) :

    # 최근일자(=기준일자 하루전)
    end_time = std_time - relativedelta(days=1)

    # 시작일자(=2년전)
    start_time = std_time - relativedelta(years=2) - relativedelta(days=1)

    ## 그래프 작성용 (=3년전)
    start_before = std_time - relativedelta(years=3) - relativedelta(days=1)

    end_time = end_time.strftime("%Y-%m-%d")
    start_time = start_time.strftime("%Y-%m-%d")
    start_before = start_before.strftime("%Y-%m-%d")

    # 분석 기간 설정
    table.index = pd.to_datetime(table.index)
    table_tmp = table[(table.index >= start_time)&(table.index <= end_time)]
    table_graph = table[(table.index >= start_before)&(table.index <= end_time)]

    # 기간 내 데이터 공백 존재시
    if len(table_tmp.index) < 700 :
        return None, None, None

    # 주별 데이터 테이블로 변환 (104주)
    tmp_list = []

    for i in range(0,104,1) :
        if i == 0 :
            days_7 = table_tmp.iloc[-(7*(i+1)) : , ].sum()
        else :
            days_7 = table_tmp.iloc[-(7*(i+1)) : -(7*(i)) , ].sum()
        tmp_list.append(days_7)
    tmp_frame = pd.DataFrame(tmp_list)

    r_idx = [i for i in range(tmp_frame.shape[0]-1,-1,-1)]
    result_tmp = pd.DataFrame(tmp_frame, index=r_idx)
    result_tmp.reset_index(drop=True, inplace=True)

    ## 그래프 작성용
    tmp_list_graph = []

    for j in range(0,156,1) :
        if j == 0 :
            days_7 = table_graph.iloc[-(7*(j+1)) : , ].sum()
        else :
            days_7 = table_graph.iloc[-(7*(j+1)) : -(7*(j)) , ].sum()
        tmp_list_graph.append(days_7)
    tmp_graph = pd.DataFrame(tmp_list_graph)

    r_idx_2 = [i for i in range(tmp_graph.shape[0]-1,-1,-1)]
    result_tmp_gph = pd.DataFrame(tmp_graph, index=r_idx_2)
    result_tmp_gph.reset_index(drop=True, inplace=True)


    # 기간에 맞춰 상대적 검색량 수정
    result_tmp = result_tmp/result_tmp.max() * 100
    result_tmp_gph = result_tmp_gph/result_tmp_gph.max() * 100

    # 인덱스 설정
    result_tmp.index = pd.date_range(end = f'{end_time}', periods = len(result_tmp.index), freq = '7d')
    result_tmp_gph.index = pd.date_range(end = f'{end_time}', periods = len(result_tmp_gph.index), freq = '7d')

    ## 1. 상승 추세 유무
    top = slop(result_tmp)              # 최근 2년간 추세
    middle = slop(result_tmp.iloc[-54:,]) # 최근 2/3지점 추세
    bottom = slop(result_tmp.iloc[-32:,]) # 최근 1/3지점 추세

    last = result_tmp.iloc[-1,0]
    last_2 = result_tmp.iloc[-2,0]

    recent = slop(result_tmp.iloc[-4:,]) # 최근 한달 추세
    recent_2 = slop(result_tmp.iloc[-7:,]) # 최근 한달 추세

    # 분산 기준
    var = np.var(table_tmp.iloc[:,0])

    # 그래프용 테이블 만들기
    result_graph = pd.DataFrame()
    result_graph['검색일자'] = result_tmp_gph.index
    result_graph['기준일자'] = today.strftime("%Y-%m-%d")
    result_graph['유형'] = '주별지속상승'
    result_graph['연관검색어'] = result_tmp_gph.columns[0]
    result_graph['검색량'] = result_tmp_gph.values

    result_graph = result_graph[['기준일자','유형', '연관검색어', '검색일자', '검색량']]

    if (0 < top) & (0 < middle) & (0 < bottom) & (last_2 < last) & (0 < recent_2 < 10) & (recent < 10) :
        print(f'주별 지속상승 키워드 발견 : {table_graph.columns[0]}')
        table_tmp_2 = result_tmp.copy()

        return table_tmp_2 , result_graph, round(top*100, 2)
    else :
        return None, None, None

### (4) 월별 급상승 키워드 선별 함수

In [ ]:
# 월별 급상승 키워드 선별 함수

def monthly_up(table, std_time) :

    # 최근일자(=기준일자 하루전)
    end_time = std_time - relativedelta(days=1)
    end_time = end_time.strftime("%Y-%m-%d")
    # 시작일자(=1년전)
    start_time = std_time - relativedelta(years=3) - relativedelta(days=1)
    start_time = start_time.strftime("%Y-%m-%d")

    # 분석 기간 설정
    table.index = pd.to_datetime(table.index)
    table_tmp = table[(table.index >= start_time)&(table.index <= end_time)]

        # 기간 내 데이터 공백 존재시
    if len(table_tmp.index) < 1080 :
        return None, None, None

    # 월별 데이터 테이블로 변환 (36개월)
    tmp_list = []

    for i in range(0,36,1) :
        if i == 0 :
            days_28 = table_tmp.iloc[-(28*(i+1)) : , ].sum()
        else :
            days_28 = table_tmp.iloc[-(28*(i+1)) : -(28*(i)) , ].sum()
        tmp_list.append(days_28)
    tmp_frame = pd.DataFrame(tmp_list)

    r_idx = [i for i in range(tmp_frame.shape[0]-1,-1,-1)]
    result_tmp = pd.DataFrame(tmp_frame, index=r_idx)
    result_tmp.reset_index(drop=True, inplace=True)


    # 기간에 맞춰 상대적 검색량 수정
    result_tmp = result_tmp/result_tmp.max() * 100

    # 인덱스 설정
    result_tmp.index = pd.date_range(end = f'{end_time}', periods = len(result_tmp.index), freq = '28d')

    # 검색량 기준
    last = result_tmp.iloc[-1,0] # 가장 최근 일자 상대적 검색량
    last_2 = result_tmp.iloc[-2,0] # 가장 최근 일자 상대적 검색량
    recent = slop(result_tmp.iloc[-3:,])

    # 분산 기준
    var = np.var(result_tmp.iloc[:,0])
    table_tmp_2 = result_tmp.copy()

    rise_rt = (last - last_2)/last_2 * 100
    rist_rt = round(rise_rt,2)

    # 그래프용 테이블 만들기
    result_graph = pd.DataFrame()
    result_graph['검색일자'] = result_tmp.index
    result_graph['기준일자'] = today.strftime("%Y-%m-%d")
    result_graph['유형'] = '월별급상승'
    result_graph['연관검색어'] = result_tmp.columns[0]
    result_graph['검색량'] = result_tmp.values

    result_graph = result_graph[['기준일자','유형', '연관검색어', '검색일자', '검색량']]

    if last == 100 :
        print(f'월별 급상승 키워드 발견 : {result_tmp.columns[0]}')

        return table_tmp_2, result_graph, rist_rt
    else :
        if (last >= last_2 * 2.0) & (last >= 60) & (recent >= 15) :

            print(f'월별 급상승 키워드 발견 : {result_tmp.columns[0]}')

            return table_tmp_2, result_graph, rist_rt

        elif (last >= 95) & (last > last_2)  & (recent >= 15) :

            print(f'월별 급상승 키워드 발견 : {result_tmp.columns[0]}')

            return table_tmp_2, result_graph, rist_rt

        elif (last_2 * 2.0 > 100) & (last >= 85) & (last > last_2) & ((last - last_2) > 5)  & (recent >= 15) :

            print(f'월별 급상승 키워드 발견 : {result_tmp.columns[0]}')

            return table_tmp_2, result_graph, rist_rt

        else :
            return None, None, None

### (5) 월별 지속상승 키워드 선별 함수

In [ ]:
# 월별 지속상승 키워드 선별 함수

def monthly_stay(table, std_time) :

    # 최근일자(=기준일자 하루전)
    end_time = std_time - relativedelta(days=1)
    end_time = end_time.strftime("%Y-%m-%d")
    # 시작일자(=1년전)
    start_time = std_time - relativedelta(years=3) - relativedelta(days=1)
    start_time = start_time.strftime("%Y-%m-%d")

    # 분석 기간 설정
    table.index = pd.to_datetime(table.index)
    table_tmp = table[(table.index >= start_time)&(table.index <= end_time)]

        # 기간 내 데이터 공백 존재시
    if len(table_tmp.index) < 1080 :
        return None, None, None

    # 월별 데이터 테이블로 변환 (36개월)
    tmp_list = []

    for i in range(0,36,1) :
        if i == 0 :
            days_28 = table_tmp.iloc[-(28*(i+1)) : , ].sum()
        else :
            days_28 = table_tmp.iloc[-(28*(i+1)) : -(28*(i)) , ].sum()
        tmp_list.append(days_28)
    tmp_frame = pd.DataFrame(tmp_list)

    r_idx = [i for i in range(tmp_frame.shape[0]-1,-1,-1)]
    result_tmp = pd.DataFrame(tmp_frame, index=r_idx)
    result_tmp.reset_index(drop=True, inplace=True)

    # 기간에 맞춰 상대적 검색량 수정
    result_tmp = result_tmp/result_tmp.max() * 100

    # 인덱스 설정
    result_tmp.index = pd.date_range(end = f'{end_time}', periods = len(result_tmp.index), freq = '28d')

    ## 1. 상승 추세 유무
    top = slop(result_tmp)              # 최근 3년간 추세
    middle = slop(result_tmp.iloc[-18:,]) # 최근 2년간 추세
    bottom = slop(result_tmp.iloc[-13:,]) # 최근 1년간 추세

    last = result_tmp.iloc[-1,0]
    last_2 = result_tmp.iloc[-2,0]

    recent = slop(result_tmp.iloc[-3:,]) # 최근 한달 추세
    recent_2 = slop(result_tmp.iloc[-5:,])
    recent_3 = slop(result_tmp.iloc[-8:,]) # 최근 두달 추세

    # 그래프용 테이블 만들기
    result_graph = pd.DataFrame()
    result_graph['검색일자'] = result_tmp.index
    result_graph['기준일자'] = today.strftime("%Y-%m-%d")
    result_graph['유형'] = '월별지속상승'
    result_graph['연관검색어'] = result_tmp.columns[0]
    result_graph['검색량'] = result_tmp.values

    result_graph = result_graph[['기준일자','유형', '연관검색어', '검색일자', '검색량']]

    if (0 < top) & (0 < middle) & (0 < bottom) & (last_2 < last) & (0 < recent < 15) & (-3 < recent_2) & (-3 < recent_3) :
        print(f'월별 지속상승 키워드 발견 : {result_tmp.columns[0]}')
        table_tmp_2 = result_tmp.copy()

        return table_tmp_2, result_graph, round(top*100, 2)

    else :
        return None, None, None

### (6) 월별 규칙성 키워드 선별 함수

In [ ]:
# 상승하는 달 검토하는 함수

def month_check(table) :

    up_month_list = []

    table_tmp = table.copy()

    table_tmp.loc[:,'year'] = table_tmp.index.year
    table_tmp.loc[:,'month'] = table_tmp.index.month
    sum_table = table_tmp.groupby(['year','month'])[[f'{table.columns[0]}']].sum() # 월별 합계
    sum_table.reset_index(inplace=True)
    sum_table.iloc[:,-1] = sum_table.iloc[:,-1]/sum_table.iloc[:,-1].max() * 100 # 상대적 검색량 변환


    for month in range(1,13) :
        if month != 12 :
            sub_result = sum_table[sum_table['month'] == month].reset_index(drop=True).iloc[:,-1] - sum_table[sum_table['month'] == month+1].reset_index(drop=True).iloc[:,-1]
        else :
            sub_result = sum_table[sum_table['month'] == 12].reset_index(drop=True).iloc[:,-1] - sum_table[sum_table['month'] == 1].reset_index(drop=True).iloc[:,-1]

        if (sub_result[1] * sub_result[2]) > 0 :
            if (sub_result[1]) < 0 :
                if month != 12 :
                    up_month_list.append(month+1)
                else :
                    up_month_list.append(1)
            else :
                pass
        else :
            pass

    return up_month_list

In [ ]:
# 월별 규칙성 키워드 선별 함수

def monthly_rule(table, std_time) :
    # 최근일자(=기준일자 하루전)
    end_time = today - relativedelta(days=1)
    end_time = end_time.strftime("%Y-%m-%d")
    # 시작일자(=3년전)
    start_time = today - relativedelta(years=3) - relativedelta(days=1)
    start_time = start_time.strftime("%Y-%m-%d")

    # 예측을 위한 기간 분리
    end_pred = today
    start_pred = end_pred - relativedelta(years=3)
    end_pred = end_pred.strftime("%Y-%m")
    start_pred = start_pred.strftime("%Y-%m")

    # 분석 기간 설정
    result.index = pd.to_datetime(result.index)
    table_tmp = result[(result.index >= start_time)&(result.index <= end_time)]
    table_pred = result[(result.index >= start_pred)&(result.index < end_pred)]


    # 기간 내 데이터 공백 존재시
    if len(table_tmp.index) < 1080 :
        return None, None, None, None

    # 월별 데이터 테이블로 변환 (36개월)
    tmp_list = []

    for i in range(0,36,1) :
        if i == 0 :
            days_28 = table_tmp.iloc[-(28*(i+1)) : , ].sum()
        else :
            days_28 = table_tmp.iloc[-(28*(i+1)) : -(28*(i)) , ].sum()
        tmp_list.append(days_28)
    tmp_frame = pd.DataFrame(tmp_list)

    r_idx = [i for i in range(tmp_frame.shape[0]-1,-1,-1)]
    result_tmp = pd.DataFrame(tmp_frame, index=r_idx)
    result_tmp.reset_index(drop=True, inplace=True)

    # 기간에 맞춰 상대적 검색량 수정
    result_tmp = result_tmp/result_tmp.max() * 100

    # 인덱스 설정
    result_tmp.index = pd.date_range(end = f'{end_time}', periods = len(result_tmp.index), freq = '28d')

    # 규칙성 검증
    ## 1년씩 확인
    year_1 = result_tmp[:12]   # 최원 1년
    year_1_new = year_1/year_1.max() * 100 # 상대적 변환

    year_2 = result_tmp[12:24]  # 이전 1년
    year_2_new = year_2/year_2.max() * 100 # 상대적 변환

    year_3 = result_tmp[24:36]  # 최근 1년
    year_3_new = year_3/year_3.max() * 100 # 상대적 변환

    dist_1 = dtw(year_1_new,year_2_new,keep_internals=True).distance
    dist_2 = dtw(year_2_new,year_3_new,keep_internals=True).distance
    dist_3 = dtw(year_1_new,year_3_new,keep_internals=True).distance

    var = np.var(result_tmp.iloc[:,0])
    var_1 = np.var(year_1_new.iloc[:,0])
    var_2 = np.var(year_2_new.iloc[:,0])
    var_3 = np.var(year_3_new.iloc[:,0])

    ## 2년씩 확인
    year_4 = result_tmp[:24]
    year_4_new = year_4/year_4.max() * 100

    year_5 = result_tmp[12:36]
    year_5_new = year_5/year_5.max() * 100

    dist_4 = dtw(year_4_new,year_5_new,keep_internals=True).distance

    ## 1. 상승 추세 유무
    total_slop = slop(result_tmp)

    or_list = [dist_1, dist_2, dist_3]
    new_list = [x for x in or_list if x < 100]

    similarity_rt = (1 - (dist_1 + dist_2)/2000) * 100
    similarity_rt = round(similarity_rt,2)

    # 그래프용 테이블 만들기
    result_graph = pd.DataFrame()
    result_graph['검색일자'] = result_tmp.index
    result_graph['기준일자'] = today.strftime("%Y-%m-%d")
    result_graph['유형'] = '월별규칙성'
    result_graph['연관검색어'] = result_tmp.columns[0]
    result_graph['검색량'] = result_tmp.values

    result_graph = result_graph[['기준일자','유형', '연관검색어', '검색일자', '검색량']]

    if len(new_list) >= 2 :
        ### 1번 조건 : 거리가 150보다 작으며, 더 큰 거리가 작은 거리에 비해 2배 미만
        if (max(dist_1, dist_2) < 110.0) & (min(dist_1, dist_2) < 100.0) & (min(dist_1, dist_2) * 2 > max(dist_1, dist_2)) & (dist_4 < 280.0) & (-0.3 < (total_slop) < 0.5):
            print(f'월별 규칙성 키워드 발견 : {result_tmp.columns[0]}')
            table_tmp_2 = result_tmp.copy()

            # 상승하는 달
            rising_month = month_check(table_pred)

            return table_tmp_2 , result_graph, similarity_rt, rising_month

        ### 2번 조건 : 거리가 150보다 하나만 크고, 2년씩 비교한 거리가 300보다 작음
        elif (max(dist_1, dist_2) < 150.0) & (dist_4 < 220.0) & (-0.3 < (total_slop) < 0.5):
            print(f'월별 규칙성 키워드 발견 : {result_tmp.columns[0]}')
            table_tmp_2 = result_tmp.copy()

            # 상승하는 달
            rising_month = month_check(table_pred)

            return table_tmp_2 , result_graph, similarity_rt, rising_month
        else :
            return None, None, None, None
    else :
        return None, None, None, None


## 2) 정보 지정

In [ ]:
# 날짜 지정
today = datetime.now(timezone('Asia/Seoul'))

# 저장할 폴더 미리 생성
os.mkdir(f'./data/tmp')
os.mkdir(f'./data/result')
os.mkdir(f'./data/result/{day}')
os.mkdir(f'./data/result/{day}/graph')

# 분석 기간 설정
start_index = (today- relativedelta(days=1)).strftime("%Y-%m-%d")
end_index = (today - relativedelta(years=3) - relativedelta(days=1)).strftime("%Y-%m-%d")

# API 아이디 불러오기
import api_id

URL = "https://openapi.naver.com/v1/datalab/search"


In [ ]:
# 결과 내용 저장 함수

def check_result(result_1, result_2, result_3, frame_1, frame_2, srch_keyword , rlt_keyword , dict_1 , dict_2 , table , dict_3, dict_4)  :
    if result_1 is None :
        pass
    else :
        if str(srch_keyword) in dict_1 :
            dict_1[str(srch_keyword)].append(str(rlt_keyword))
        else :
            dict_1[str(srch_keyword)] = [str(rlt_keyword)]
        dict_2[str(rlt_keyword)] = table[table['연관키워드'] == rlt_keyword]['월간검색수_합계'].values[0]
        dict_3[str(rlt_keyword)] = result_3
        dict_4[str(rlt_keyword)] = table['연관키워드'][:10].to_list()
        frame_1 = pd.concat([frame_1, result_1], axis = 1)
        frame_2 = pd.concat([frame_2, result_2], axis = 0)
    return dict_1, dict_2, dict_3, dict_4, frame_1, frame_2

## 3) 유형 분류

In [ ]:
daily_up_trend = pd.DataFrame()
weekly_up_trend = pd.DataFrame()
weekly_stay_trend = pd.DataFrame()
monthly_up_trend = pd.DataFrame()
monthly_stay_trend = pd.DataFrame()
monthly_rule_trend = pd.DataFrame()

graph_table = pd.DataFrame()
graph_table_day = pd.DataFrame(index = pd.date_range(start = start_index, end = end_index, freq = '1d'))
graph_table_week = pd.DataFrame(index = pd.date_range(start = start_index, end = end_index, freq = '7d'))
graph_table_month = pd.DataFrame(index = pd.date_range(start = start_index, end = end_index, freq = '28d'))

dict_kind = {}  # 검색 키워드 구분자
dict_srch = {}  # 연관검색어와 그에 해당한 연관검색어
dict_rl = {}    # 연관검색어 검색량 정보
daily_up_info = {}
weekly_up_info = {}
weekly_stay_info = {}
monthly_up_info = {}
monthly_stay_info = {}
monthly_rule_info = {}

check_list = [] # 키워드 저장용

up_month = {} # 상승하는 달


cnt = 1   # 트렌드 불러온 횟수


for name in main_keyword.keyword_final :

    ceil = 50 # 최대 연관검색어 수

    df_table = pd.read_csv('./data/rl_srch/' + str(day) + f'/{name}.csv', encoding='euc-kr')

    print(f'################################################ {name} ################################################')

    times = 0

    if cnt <= 1000 :
        ID = api_id.id_1['client_id']
        PW = api_id.id_1['client_secret']
        id_num = 1
    else :
        pass

    while times < 50 :

        nm = df_table['연관키워드'][times]

        # 키워드 중복 확인
        while nm in check_list :

          if nm in dict_rl.keys() :
            if str(name) in dict_kind :
                dict_kind[str(name)].append(str(nm))
            else :
                dict_kind[str(name)] = [str(nm)]
          else :
            pass
          ceil = ceil + 1
          nm = df_table['연관키워드'][ceil]

        # print(nm)
        result = trend_load(today, str(nm), ID, PW, URL)
        result.to_csv(f'./data/tmp/{nm}_result.csv',encoding='utf-8-sig')


        if len(result.index) == 0 :
            cnt = cnt + 1
            continue

        # 일별 급상승 검토
        tmp, tmp_gph, tmp_info = daily_up(result,today)
        dict_kind, dict_rl, daily_up_info, dict_srch, daily_up_trend, graph_table_day = check_result(tmp, tmp_gph, tmp_info, daily_up_trend, graph_table_day, name, nm, dict_kind, dict_rl, df_table, daily_up_info ,dict_srch)

        # 주별 급상승 검토
        tmp_2, tmp_gph_2, tmp_info_2 = weekly_up(result,today)
        dict_kind, dict_rl, weekly_up_info, dict_srch, weekly_up_trend, graph_table_week = check_result(tmp_2, tmp_gph_2, tmp_info_2, weekly_up_trend, graph_table_week, name, nm, dict_kind, dict_rl, df_table, weekly_up_info , dict_srch)

        # 주별 지속상승 검토
        tmp_3, tmp_gph_3, tmp_info_3 = weekly_stay(result, today)
        dict_kind, dict_rl, weekly_stay_info, dict_srch, weekly_stay_trend, graph_table_week = check_result(tmp_3, tmp_gph_3, tmp_info_3, weekly_stay_trend, graph_table_week, name, nm, dict_kind, dict_rl, df_table, weekly_stay_info, dict_srch)

        # 월별 급상승 검토
        tmp_4, tmp_gph_4, tmp_info_4 = monthly_up(result, today)
        dict_kind, dict_rl, monthly_up_info, dict_srch, monthly_up_trend, graph_table_month = check_result(tmp_4, tmp_gph_4, tmp_info_4, monthly_up_trend, graph_table_month, name, nm, dict_kind, dict_rl, df_table, monthly_up_info, dict_srch)

        # 월별 지속상승 검토
        tmp_5, tmp_gph_5, tmp_info_5 = monthly_stay(result, today)
        dict_kind, dict_rl, monthly_stay_info, dict_srch, monthly_stay_trend, graph_table_month = check_result(tmp_5, tmp_gph_5, tmp_info_5, monthly_stay_trend, graph_table_month, name, nm, dict_kind, dict_rl, df_table, monthly_stay_info , dict_srch)

        # 월별 규칙성 증감 검토
        tmp_6, tmp_gph_6, tmp_info_6, up_months = monthly_rule(result, today)
        dict_kind, dict_rl, monthly_rule_info, dict_srch, monthly_rule_trend, graph_table_month = check_result(tmp_6, tmp_gph_6, tmp_info_6, monthly_rule_trend, graph_table_month, name, nm, dict_kind, dict_rl, df_table, monthly_rule_info , dict_srch)

        if tmp_6 is None :
            pass
        else :
            if str(nm) in up_month :
                pass
            else :
                up_month[str(nm)] = up_months

        check_list.append(nm)
        times = times + 1
        cnt = cnt + 1
        if cnt % 1000 == 0 :
            id_num  = id_num + 1
            print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
            print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ API 변경 @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
            print(f'@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ {id_num}번째 ID 사용 @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
            print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
            ID = eval('api_id.id_' + str(id_num))['client_id']
            PW = eval('api_id.id_' + str(id_num))['client_secret']


################################################ 주식 ################################################
주별 지속상승 키워드 발견 : CMA
주별 급상승 키워드 발견 : ETF
월별 급상승 키워드 발견 : 양도세
월별 급상승 키워드 발견 : 국내주식
주별 급상승 키워드 발견 : 한국증시거래시간
월별 급상승 키워드 발견 : 한국증시거래시간
월별 규칙성 키워드 발견 : 엑셀
주별 급상승 키워드 발견 : 주식개장시간
월별 급상승 키워드 발견 : 주식개장시간
월별 급상승 키워드 발견 : 오늘주식시장
월별 급상승 키워드 발견 : PF
주별 지속상승 키워드 발견 : 반도체관련주
월별 급상승 키워드 발견 : 증시휴장일
월별 급상승 키워드 발견 : 주식장
월별 급상승 키워드 발견 : IRP계좌개설
주별 지속상승 키워드 발견 : CMA통장
월별 급상승 키워드 발견 : 배당주
주별 급상승 키워드 발견 : CMA금리비교
주별 지속상승 키워드 발견 : CMA금리비교
월별 급상승 키워드 발견 : CMA금리비교
월별 지속상승 키워드 발견 : CMA금리비교
################################################ 금리 ################################################
주별 급상승 키워드 발견 : 적금계산기
월별 급상승 키워드 발견 : 적금계산기
주별 급상승 키워드 발견 : 파킹통장
월별 급상승 키워드 발견 : 파킹통장
주별 지속상승 키워드 발견 : 예금이자높은은행
주별 급상승 키워드 발견 : 저축은행파킹통장
주별 지속상승 키워드 발견 : 저축은행파킹통장
월별 급상승 키워드 발견 : 저축은행파킹통장
주별 지속상승 키워드 발견 : 금리인하
월별 급상승 키워드 발견 : 금리인하
################################################ 금융상품 ###########################################

In [ ]:
# 분류 결과 저장을 위한 딕셔너리 리스트
## 패턴 유형, 검색량, 연관검색어, 유형별
dict_list = ['dict_kind', 'dict_srch', 'dict_rl', 'daily_up_info', 'weekly_up_info', 'weekly_stay_info', 'monthly_up_info', 'monthly_stay_info', 'monthly_rule_info', 'up_month']

# 유형별 영문&한글 이름
trend_type = dict(zip(['daily_up_trend', 'weekly_up_trend', 'weekly_stay_trend', 'monthly_up_trend', 'monthly_stay_trend', 'monthly_rule_trend'],
                        ['일별 급상승', '주별 급상승', '주별 지속상승', '월별 급상승', '월별 지속상승', '월별 규칙성']))

# 결과 저장을 위한 폴더 생성
os.mkdir('./data/dict')
os.mkdir('./data/types')
os.mkdir('./data/json')

# 유형 분류 결과 저장
for k in dict_list :
    with open(f'./data/dict/{k}.pkl','wb') as f :
        pickle.dump(eval(f'{k}'), f)

for type_eng, type_kor in trend_type.items() :
    eval(type_eng).to_csv(f'./data/types/{type_eng}_result.csv',encoding='utf-8-sig')

#  3. 상세분석

In [ ]:
# 네이버 뉴스 및 블로그 수집 API 정보
client_id = "ByXmMvAqMIxyVUY_h17L"
client_secret = "2x7yByvNSN"

## 1) 구글 급상승 검색어

In [ ]:
# 구글 급상승 연관검색어 수집을 위한 함수
def google(word) :
    time.sleep(1.0)
    pytrends = TrendReq(hl='ko-KR', tz=540,   retries=2)

    # 키워드 입력
    kw_list = [f'{word}']
    time.sleep(0.5)

    # 기간 입력
    pytrends.build_payload(kw_list, geo='KR', timeframe='today 1-m', cat=0, gprop='')

    try :
        tmp = pytrends.related_queries()

        if tmp[f'{word}']['rising'] is None :
            df_RelativeKW_rising = []
        else :
            df_RelativeKW_rising = list(tmp[f'{word}']['rising']['query'])

    except :
        df_RelativeKW_rising = google(word)
        pass
    finally :
        return df_RelativeKW_rising

# 구글 급상승 연관검색어 결과 저장 함수
def goo_result(types) :

    top_list = []

    length = len(types)

    for i in range(0,length) :
        word = types[i]
        rising_keyword = google(word)
        top_list.append(rising_keyword)

    return top_list

## 2) 네이버 뉴스 수집
* 선별된 패턴의 검색어를 입력해 최근 일자부터 10개 뉴스의 제목, 본문, 링크 수집

In [ ]:
## api 수집 코드

def news(name) :
        # [CODE 1]
    def getRequestUrl(url):
        req = urllib.request.Request(url)
        req.add_header("X-Naver-Client-Id", client_id)
        req.add_header("X-Naver-Client-Secret", client_secret)

        try:
            response = urllib.request.urlopen(req)
            if response.getcode() == 200:
                return response.read().decode('utf-8')
        except Exception as e:
            return None

    # [CODE 2]
    def getNaverSearch(node, srcText, start, display):
        base = "https://openapi.naver.com/v1/search"
        node = "/%s.json" % node
        parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)

        url = base + node + parameters
        responseDecode = getRequestUrl(url)  # [CODE 1]

        if (responseDecode == None):
            return None
        else:
            return json.loads(responseDecode)


    # [CODE 3]
    def getPostData(post, jsonResult, cnt):
        title = post['title']
        description = post['description']
        org_link = post['originallink']
        link = post['link']

        pDate = datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
        pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')

        jsonResult.append({'cnt': cnt, 'title': title, 'description': description,
                           'org_link': org_link, 'link': org_link, 'pDate': pDate})
        return

    # [CODE 0]
    def main():
        node = 'news'  # 크롤링 할 대상 (뉴스)
        srcText = f'{name}'
        cnt = 0
        jsonResult = []

        jsonResponse = getNaverSearch(node, srcText, 1, 100)  # [CODE 2]
        total = jsonResponse['total']

        while ((jsonResponse != None) and (jsonResponse['display'] != 0)):
            for post in jsonResponse['items']:
                cnt += 1
                getPostData(post, jsonResult, cnt)  # [CODE 3]

            start = jsonResponse['start'] + jsonResponse['display']
            jsonResponse = getNaverSearch(node, srcText, start, 100)  # [CODE 2]

        with open('./data/json/%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
            jsonFile = json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)

            outfile.write(jsonFile)

        return json.loads(jsonFile)

    if __name__ == '__main__':
        main()

    with open(f'./data/json/{name}_naver_news.json', 'r', encoding='UTF8') as f:

        json_data = json.load(f)

    return json_data

In [ ]:
# 검색어별 뉴스 제목, 본문, 링크 추출 코드
def news_result(name) :

    title = [] # 제목
    desc = []  # 본문
    link = []  # 링크
    index_list = []

    for i in range(0,len(name),1) :

        sr_name = name[i]

        tmp_title = []
        tmp_link = []

        json_data = news(sr_name)

        j = 0
        if len(json_data) < 10 :
          count = len(json_data)
        else :
          count = 10

        # 검색어당 뉴스 10개씩 결과 테이블 적재
        while j < count :

            # 뉴스 제목 중 광고성 또는 활용도가 떨어지는 뉴스 제외
            if ('운세' in json_data[j]['title']) or ('칼럼' in json_data[j]['title']) or('인사' in json_data[j]['title']):
                count = count + 1
                j = j + 1
                continue

            # 뉴스 제목 (+ json 코드 변경)
            news_title = json_data[j]['title'].replace('<b>', ' ').replace('</b>', ' ').replace('&quot;', '"').replace('&apos;', '\'').replace('amp;', '').replace('&lt;', '<').replace('&gt;', '>')

            # 뉴스 제목이 동일한 경우 다른 뉴스 수집
            if news_title in title :
                count = count + 1
                j = j + 1
                continue

            # 기존에 수집한 뉴스와 유사한 제목인 경우 다른 뉴스 수집
            for tle in title :
              ratio = SequenceMatcher(None, news_title, tle).ratio()*100
              if ratio >= 80 :
                count = count + 1
                j = j +1
                continue

            # 뉴스 제목
            tmp_title.append(news_title)

            # 뉴스 본문 수집 (현재는 분석 결과 대상에서 빠졌지만 나중에 사용할 수도 있어서 남겨둠) (+ json 코드 변경)
            news_desc = json_data[j]['description'].replace('<b>', ' ').replace('</b>', ' ').replace('&quot;', '"').replace('&apos;', '\'').replace('amp;', '').replace('&lt;', '<').replace('&gt;', '>')
            desc.append(news_desc)

            # 뉴스 링크
            news_link = json_data[j]['link']
            tmp_link.append(news_link)

            # 뉴스 인덱스
            index_list.append(count-j)

            j = j + 1

        title.append(tmp_title)
        link.append(tmp_link)



    return title, desc, link, index_list

## 3) 네이버 블로그 수집
* 선별된 패턴의 검색어를 입력해 최근 일자부터 10개 뉴스의 제목, 본문, 링크 수집

In [ ]:
# 검색어별 관련 블로그 수집 함수
def blog(client_id,client_secret,query,display,start,sort):
    encText = urllib.parse.quote(query)
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText + \
    "&display=" + str(display) + "&start=" + str(start) + "&sort=" + sort

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        response_json = json.loads(response_body)
    else:
        print("Error Code:" + rescode)

    return pd.DataFrame(response_json['items'])


# 최근 1년간 검색어와 관련된 블로그 수집
def blog_result(types, std_time) :

    # 최근일자(=기준일자 하루전)
    end_time = std_time - relativedelta(days=1)
    # 시작일자(=2년전)
    start_time = std_time - relativedelta(weeks=1) - relativedelta(days=1)
    end_time = end_time.strftime("%Y-%m-%d")
    start_time = start_time.strftime("%Y-%m-%d")

    display=100
    start=1
    sort='date'

    blog_active = []

    leng = len(types)

    for i in range(0,leng) :
        query = f'{types[i]}'

        result_all=pd.DataFrame()
        for i in range(0,10):
            start= 1 + 100*i
            time.sleep(0.1)
            result= blog(client_id,client_secret,query,display,start,sort)

            result_all=pd.concat([result_all,result])
        result_all.reset_index(inplace=True, drop = True)

        result_all['postdate'] = pd.to_datetime(result_all['postdate'])

        length = len(result_all[(result_all['postdate'] >= start_time) & ((result_all['postdate'] <= end_time))].index)

        # 활동성 (1000개 중 최근 일주일동안 블로그 발간 비율)
        result = round(length/1000 * 100,3)

        blog_active.append(result)

    return blog_active


# 4.결과테이블

## 1) 결과 테이블 생성 함수

### (1) 테이블 생성

In [ ]:
# 결과 테이블 생성 함수
def make_result(types, std_time, kor) :

    # 뉴스 제목, 본문, 링크
    title, desc, link, idx = news_result(types)

    # 구글 검색어
    # goo = goo_result(types)

    # 블로그 활동성
    blog = blog_result(types, std_time)

    # 결과 저장 데이터 프레임
    result = pd.DataFrame()

    length = len(types)

    for i in range(0,length) :
        try:

            test = pd.DataFrame()

            # 기준 일자 (작동을 안함..)
            test['기준일자'] = std_time.strftime("%Y-%m-%d")

            # 네이버 뉴스 - 제목
            ## 10개가 안될 경우 빈칸으로 기입
            if idx[10*i] != 10 :
                nan_list = [' '] * (10 - len(title[i]))
                new_list = title[i] + nan_list
                test['뉴스제목'] = new_list
            else :
                test['뉴스제목'] = title[i]

            # 네이버 뉴스 - 링크
            ## 10개가 안될 경우 빈칸으로 기입
            if idx[10*i] != 10 :
                nan_list = [' '] * (10 - len(title[i]))
                new_list = link[i] + nan_list
                test['뉴스링크'] = new_list
            else :
                test['뉴스링크'] = link[i]

            # 구글 검색어
            ##  10개가 안될 경우 빈칸으로 기입
            # if len(goo[i][:10]) != 10 :
            #     nan_list = [' '] * (10 - len(goo[i][:10]))
            #     new_list = goo[i][:10] + nan_list
            #     test['구글검색어'] = new_list
            # else :
            #     test['구글검색어'] = goo[i][:10]
            test['구글검색어'] = [' '] * 10

            # 네이버 검색어
            ## 10개가 안될 경우 빈칸 기입
            if len(dict_srch[f'{types[i]}']) != 10 :
                nan_naver = [' '] * (10 - len(dict_srch[f'{types[i]}']))
                list_new = dict_srch[f'{types[i]}'] + nan_naver
                test['네이버검색어'] = list_new
            else :
                test['네이버검색어'] = dict_srch[f'{types[i]}']

            # 네이버 블로그를 통한 활동성
            test['활동성'] = f'{blog[i]}%'

            # 검색 패턴 유형
            test['유형'] = kor

            # 연관검색어
            test['연관검색어'] = types[i]

            # 연관검색어의 최근 4주 검색량
            test['검색량'] = dict_rl[f'{types[i]}']

            # 유형별 지표 데이터
            test['지표'] = str(eval(str(type_eng[:-5]) + 'info')[f'{types[i]}']) + '%'

            # 검색키워드
            test['검색키워드'] = str([[keys for keys, items in dict_kind.items() if f'{types[i]}' in items]]).replace('\'', '').replace('[', '').replace(']','')

            # 상승월
            ## 월별 규칙성인 경우 상승이 예상되는 월 기입
            if kor == '월별 규칙성' :
                if len(up_month[f'{types[i]}']) != 10 :
                    nan_month = [' '] * (10 - len(up_month[f'{types[i]}']))

                    if len(up_month[f'{types[i]}']) != 0 :
                        up_month[f'{types[i]}'].sort()
                        list_month = up_month[f'{types[i]}'] + nan_month
                        test['상승월'] = list_month
                    else :
                        list_month = nan_month
                        test['상승월'] = list_month
                else :
                    up_month[f'{types[i]}'].sort()
                    test['상승월'] = up_month[f'{types[i]}']
            else :
                test['상승월'] = ' '

            # 최종 결과 합치기
            result = pd.concat([result,test], ignore_index=True)  # 결과 합치기
        except:
            print()
            continue

    # 컬럼 순서 변경
    result = result[['기준일자', '유형', '연관검색어', '검색키워드', '검색량', '지표', '뉴스제목','뉴스링크', '활동성','구글검색어', '네이버검색어', '상승월']]

    return result

### (2) 진행상황

In [ ]:
# 결과 테이블 진행 상황 표기 함수
def make_table(key, item, std_day) :

    print(f'{item} START')
    tables = eval(key)
    trends = tables.columns.to_list()

    if len(trends) > 0 :
        result_trend = make_result(trends, std_day, f'{item}')
        time.sleep(0.5)
        print(f'{item} COMPLETE')

        return result_trend

    else :
        emtys = pd.DataFrame()
        print(f'{item} PASS')
        return emtys

## 2) 정보 결과 테이블 생성

In [ ]:
# 전송용 결과 테이블 생성 함수

def make_csv(table) :

  # 컬럼 추출
  col_a = ''
  col_b = ''

  for col in table.columns :
    col_a = str(col) + '|||'
    col_b = col_b + col_a
  col_b


  # 행 추출
  row_list = []

  for j in range(0, len(table)) :
    tmp_a = ''
    tmp_b = ''

    for i in range(0, len(table.columns)) :
      tmp_a = str(table.iloc[j,i]) + '|||'
      tmp_b = tmp_b + tmp_a
    row_list.append(tmp_b)

  row_list.insert(0,col_b)
  df = pd.DataFrame(row_list)

  return df


In [ ]:
trend_type_1 = dict(zip(['daily_up_trend', 'weekly_up_trend', 'weekly_stay_trend'],
                        ['일별 급상승', '주별 급상승', '주별 지속상승']))

trend_type_2 = dict(zip(['monthly_up_trend', 'monthly_stay_trend', 'monthly_rule_trend'],
                        ['월별 급상승', '월별 지속상승', '월별 규칙성']))

### (1) 단기 결과 테이블 생성

In [ ]:
# 일별, 주별 결과 테이블 생성

today = datetime.today().astimezone(timezone('Asia/Seoul'))
result_tbl_1 = pd.DataFrame()

for type_eng, type_kor in trend_type_1.items() :

    eval(type_eng).to_csv(f'./data/types/{type_eng}_result.csv',encoding='utf-8-sig')
    table_re = make_table(str(type_eng), str(type_kor), today)
    if len(table_re.index) == 0  :
        continue
    else :
        result_tbl_1 = pd.concat([result_tbl_1,table_re])
        result_tbl_1.to_csv('./data/result/' + str(day) + '/info_result_bf.csv',encoding='utf-8-sig')

result_tbl_1.reset_index(inplace = True, drop = True)
result_tbl_1['기준일자'] = today.strftime("%Y-%m-%d")

일별 급상승 START
일별 급상승 COMPLETE
주별 급상승 START
주별 급상승 COMPLETE
주별 지속상승 START
주별 지속상승 COMPLETE


### (2) 장기 결과 테이블 생성

In [ ]:
# 월별 결과 테이블 생성

result_tbl_2 = pd.DataFrame()

for type_eng, type_kor in trend_type_2.items() :

    eval(type_eng).to_csv(f'./data/types/{type_eng}_result.csv',encoding='utf-8-sig')
    table_re = make_table(str(type_eng), str(type_kor), std_day = today)
    if len(table_re.index) == 0  :
        continue
    else :
        result_tbl_2 = pd.concat([result_tbl_2,table_re])
        result_tbl_2.to_csv('./data/result/' + str(day) + '/info_result_af.csv',encoding='utf-8-sig')

result_tbl_2.reset_index(inplace = True, drop = True)
result_tbl_2['기준일자'] = today.strftime("%Y-%m-%d")

월별 급상승 START

월별 급상승 COMPLETE
월별 지속상승 START
월별 지속상승 COMPLETE
월별 규칙성 START
월별 규칙성 COMPLETE


In [ ]:
# 결과 저장
result_tbl = pd.concat([result_tbl_1,result_tbl_2],axis=0)
result_tbl.reset_index(inplace = True, drop = True)
result_tbl.to_csv('./data/result/' + str(day) + f'/info_result_in_{day}.csv',encoding='utf-8-sig', index=False)

result_csv = make_csv(result_tbl)
result_csv.to_csv('./data/result/' + str(day) + f'/info_result_out_{day}.csv',encoding='utf-8-sig', index=False, header=False)

result_tbl

,기준일자,유형,연관검색어,검색키워드,검색량,지표,뉴스제목,뉴스링크,활동성,구글검색어,네이버검색어,상승월
0,2024-01-09,일별 급상승,전세대출계산기,채권금리,9810.0,26.06%,"""내게 맞는 상품은?""…핀크, '주택담보 대출 알아보기' 서비스 오픈",https://www.etoday.co.kr/news/view/2319267,1.5%,,환율,
1,2024-01-09,일별 급상승,전세대출계산기,채권금리,9810.0,26.06%,"핀크, ‘주택담보 대출 알아보기’ 서비스 오픈",https://www.ebn.co.kr/news/view/1607906/?sc=Naver,1.5%,,이자계산기,
2,2024-01-09,일별 급상승,전세대출계산기,채권금리,9810.0,26.06%,"핀크, ‘주택담보 대출 알아보기’ 시작",https://www.ekn.kr/web/view.php?key=2024010901...,1.5%,,에코프로주가,
3,2024-01-09,일별 급상승,전세대출계산기,채권금리,9810.0,26.06%,"핀크, ‘주택담보 대출 알아보기’ 서비스 오픈",http://news.heraldcorp.com/view.php?ud=2024010...,1.5%,,코스닥,
4,2024-01-09,일별 급상승,전세대출계산기,채권금리,9810.0,26.06%,"핀크, ‘주담대 알아보기’ 서비스 오픈…“빠르게 조회하고 바로 신청”",http://www.edaily.co.kr/news/newspath.asp?news...,1.5%,,증시,
...,...,...,...,...,...,...,...,...,...,...,...,...
4685,2024-01-09,월별 규칙성,종합소득세율,증여,12260.0,93.31%,"직방, 올해 부동산 제도는 '청년·결혼·출산 지원' 다수 신설이 핵심",https://www.ekn.kr/web/view.php?key=2024010801...,91.9%,,증여세면제한도,
4686,2024-01-09,월별 규칙성,종합소득세율,증여,12260.0,93.31%,연금 원할때…연말 소득 공제 받을까? 비과세 혜택을 받을까?,https://kookbang.dema.mil.kr/newsWeb/20240109/...,91.9%,,상속세,
4687,2024-01-09,월별 규칙성,종합소득세율,증여,12260.0,93.31%,2024년 달라질 부동산 주요 제도는,http://www.globalepic.co.kr/view.php?ud=202401...,91.9%,,중대재해처벌법,
4688,2024-01-09,월별 규칙성,종합소득세율,증여,12260.0,93.31%,"[로컬거버넌스] 경기 부천시, 새해 달라지는 제도·시책 발표",https://www.siminilbo.co.kr/news/newsview.php?...,91.9%,,양도소득세계산기,


In [ ]:
# header = result_csv.iloc[0]
# data_csv = result_csv[1:]
# file_num = len(data_csv) // 4999 + 1
# for i in range(file_num):
#   tmp_df = data_csv[i*4999:(i+1)*4999]
#   tmp_df = pd.concat([header, tmp_df])
#   tmp_df.to_csv('./data/result/' + str(day) + f'/info_result_out_{day}_{i+1}.csv',encoding='utf-8-sig', index=False, header=False)

### 3) 그래프 결과 테이블 생성

In [ ]:
# 그래프 작성용 테이블 생성
graph_table = pd.concat([graph_table_day, graph_table_week, graph_table_month])
graph_table.reset_index(drop=True, inplace=True)
graph_table.to_csv('./data/result/' + str(day) + f'/graph_result_in_{day}.csv',encoding='utf-8-sig', index=False)

graph_csv = make_csv(graph_table)
graph_csv.to_csv('./data/result/' + str(day) + f'/graph_result_out_{day}.csv',encoding='utf-8-sig', index=False, header=False)

graph_table

,기준일자,유형,연관검색어,검색일자,검색량
0,2024-01-09,일별급상승,전세대출계산기,2021-01-08,42.696620
1,2024-01-09,일별급상승,전세대출계산기,2021-01-09,37.265910
2,2024-01-09,일별급상승,전세대출계산기,2021-01-10,42.134830
3,2024-01-09,일별급상승,전세대출계산기,2021-01-11,61.797750
4,2024-01-09,일별급상승,전세대출계산기,2021-01-12,56.179770
...,...,...,...,...,...
53876,2024-01-09,월별규칙성,종합소득세율,2023-09-18,23.883701
53877,2024-01-09,월별규칙성,종합소득세율,2023-10-16,19.822412
53878,2024-01-09,월별규칙성,종합소득세율,2023-11-13,26.297394
53879,2024-01-09,월별규칙성,종합소득세율,2023-12-11,31.874853


In [ ]:
# header = graph_csv.iloc[0]
# data_csv = graph_csv[1:]
# file_num = len(data_csv) // 4999 + 1
# for i in range(file_num):
#   tmp_df = data_csv[i*4999:(i+1)*4999]
#   tmp_df = pd.concat([header, tmp_df])
#   tmp_df.to_csv('./data/result/' + str(day) + f'/graph_result_out_{day}_{i+1}.csv',encoding='utf-8-sig', index=False, header=False)

# 5.모니터링

In [ ]:
def result_list(table, trend) :
  tmp = table.copy()
  tmp_dict = dict()
  word_list = []

  for key, value in trend.items() :
    tmp_ft = tmp[tmp['유형'] == value].copy()
    if len(tmp_ft) == 0 :
      tmp_word = ''
    else :
      tmp_word = tmp_ft.sort_values('지표',ascending=False,ignore_index=True)['연관검색어'][0]
    word_list.append(tmp_word)
    rl_word = list(set(tmp_ft['연관검색어']))
    tmp_dict[f'{value}'] = rl_word
    tmp_frame = pd.DataFrame.from_dict(tmp_dict, orient='index').T

  return tmp_frame, word_list

In [ ]:
# 지난 시점 분석 결과

yest = '240102' ######## 비교 시점 기입 장소 !! ##########

result_yest = pd.read_csv(f'./info_result_in_{yest}.csv',encoding='utf-8-sig')
yest_table, yest_word = result_list(result_yest, trend_type)

# 금일 분석 결과
recent_table, recent_word = result_list(result_tbl, trend_type)



In [ ]:
inx = [f'{yest} 검색어 수', f'{day} 검색어 수', '검색어 증감', '검색어 증감(%)', '새로운 검색어', '빠진 검색어', '주요 검색어']

bf_list = []
af_list = []
diff_list = []
diff_list_2 = []
diff_list_3 = []
diff_list_4 = []

times = len(trend_type.values())

for tm in range(0,times) :

  # 식별된 검색어 개수 변화
  bf = len(yest_table.iloc[:,tm].dropna(axis = 0))
  af = len(recent_table.iloc[:,tm].dropna(axis = 0))

  diff = af - bf
  if bf == 0 :
    diff_2 = 0
  else :
    diff_2 = round((af-bf)/bf * 100,2)
  # print(diff, diff_2)

  bf_list.append(bf)
  af_list.append(af)
  diff_list.append(diff)
  diff_list_2.append(diff_2)

  # 식별된 검색어 변화
  bf_2 = list(yest_table.iloc[:,tm].dropna(axis = 0))
  af_2 = list(recent_table.iloc[:,tm].dropna(axis = 0))

  diff_3 = list(set(af_2) - set(bf_2)) # 새로 포착된 검색어
  diff_4 = list(set(bf_2) - set(af_2)) # 사라진 검색어

  # print(diff_3, diff_4)
  diff_list_3.append(diff_3)
  diff_list_4.append(diff_4)

diff_dict = dict(zip(inx,[bf_list, af_list, diff_list,diff_list_2,diff_list_3, diff_list_4, recent_word]))
diff_table = pd.DataFrame.from_dict(diff_dict,columns = trend_type.values(), orient='index')

for col in range(0,len(diff_table.columns)) :
  diff_table.iloc[4,col] = str(diff_table.iloc[4,col]).replace('[','').replace(']','')
  diff_table.iloc[5,col] = str(diff_table.iloc[5,col]).replace('[','').replace(']','')

diff_table.to_csv('./data/result/' + str(day) + f'/monitor_in_{day}.csv',encoding='utf-8-sig')

diff_tmp = diff_table.reset_index()
diff_csv = make_csv(diff_tmp)
diff_csv.to_csv('./data/result/' + str(day) + f'/monitor_out_{day}.csv',encoding='utf-8-sig', index=False, header=False)

diff_table

,일별 급상승,주별 급상승,주별 지속상승,월별 급상승,월별 지속상승,월별 규칙성
240102 검색어 수,0,55,52,108,54,79
240109 검색어 수,14,94,93,107,71,90
검색어 증감,14,39,41,-1,17,11
검색어 증감(%),0,70.91,78.85,-0.93,31.48,13.92
새로운 검색어,"'부수입', '베트남시간', '아파트실거래가조회', '쿠자', '블로그수익화', '...","'자동차보험비교', '주식자동매매', '영통역', '일수계산', '실손보험가입', ...","'대만환율', '반도체ETF', '액셀러레이터', '암보험', '베트남시간', '종...","'반도체ETF', '자동차보험가입증명서', '삼성우선주', '주식시작', '홍콩달러...","'신혼부부', '노후준비', '홍콩달러환율', '태국환율', '비상금대출', '시니...","'겨울나무', '라이나생명', '암보험', '목업', '보험해지환급금', 'KB손해..."
빠진 검색어,,"'심혈관보험', '모바일신용카드', '선부동맛집', '퇴직연금계좌', '삼성전자주식...","'심혈관보험', '기업은행배당금', '솔브케어', '비트코인ETF', '보험금청구서...","'HSCEI지수', '개인연금소득공제', '금값시세1돈', '부가세계산기', '금1...","'다모아보험', '범일동맛집', '자녀증여한도', 'HSCEI지수', '카드발급',...","'재산명시신청', '금형제작', '노후준비', '보험대출', '대여금', '0059..."
주요 검색어,1억투자,ETF,대만달러,노후자격증,공모전사이트,종합소득세신고


In [ ]:
# header = diff_csv.iloc[0]
# data_csv = diff_csv[1:]
# file_num = len(data_csv) // 4999 + 1
# for i in range(file_num):
#   tmp_df = data_csv[i*4999:(i+1)*4999]
#   tmp_df = pd.concat([header, tmp_df])
#   tmp_df.to_csv('./data/result/' + str(day) + f'/monitor_out_{day}_{i+1}.csv',encoding='utf-8-sig', index=False, header=False)

In [ ]:
import zipfile

file_path = './data/result/'

zip_file = zipfile.ZipFile(file_path + f"/output_{day}.zip", "w")
for (path, dir, files) in os.walk(file_path):
  for file in files:
    zip_file.write(os.path.join(path, file), compress_type=zipfile.ZIP_DEFLATED)

zip_file.close()

In [ ]:
from google.colab import files
files.download(f'./data/result/output_{day}.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print('외부 트렌드 분석 완료')
print('다운로드 폴더에 생성된 파일을 확인하세요')

외부 트렌드 분석 완료
다운로드 폴더에 생성된 파일을 확인하세요
